# Voronoi Diagrams

Vornoi Diagrams are a powerful computational geometry tool for analysing metric spaces. They are used to understand how KNN algorithm works. They are also used in manifold learning. They are also used in tradtional AI based planners for navigation.

In [132]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
%matplotlib qt

Defining a Utility Function to set aspect ratio of the plot

In [133]:
def forceAspect(ax,aspect=1):
    ax.set_aspect('equal')

Create an array of random points for the sake of data

In [134]:
x= np.random.rand(100,1)
y= np.random.rand(100,1)

Plotting the random point matrix

In [135]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(x,y,c="orange")

## Vornoi diagram: Notes

*[Notes on Vornoi Diagrams](https://cs.brown.edu/courses/cs252/misc/resources/lectures/pdf/notes09.pdf)

In [129]:
#plotting the random point matrix
ax.scatter(x,y,c="orange")
#pick the first point to plot the voronoi diagram of
ax.scatter(x[0],y[0],c="blue")
plt.show()
#define a global variable to store the euclidean distance of the all the points in the set S with respect to s
euc_dis = np.zeros(np.size(x)-1)

In [121]:
def compute_delimiter_slope_midpoint(p_x1, p_y1, p_x2, p_y2):
    midpoint = tuple((float((p_x1 + p_x2)/2), float((p_y1 + p_y2)/2)))
    #print(p_x1, p_y1, midpoint[0], midpoint[1], p_x2,p_y2)
    slope = (p_y2-p_y1)/(p_x2-p_x1)
    return (slope,midpoint)

In [122]:
#plotting perpendicular bisectors wrt the chosen point
def plot_perpendicular_bisector(ax,p_x, p_y, x_list, y_list):
    #plotting the random point matrix
    ax.scatter(x_list,y_list,c="orange")
    #pick the first point to plot the voronoi diagram of
    plt.scatter(p_x,p_y,c="blue")
    for i in range(1,len(x_list)):
        #compute slope of the delimiter
        m, xy=compute_delimiter_slope_midpoint(p_x, p_y, x_list[i],y_list[i])
        #print(xy)
        #ax.scatter(xy[0],xy[1], c="red")
        #ax.axline(xy, slope=m, color="black", linestyle="solid", linewidth=0.2)
        ax.axline(xy, slope=-1/m, color="green", linestyle="solid", linewidth=0.2)
    forceAspect(ax,aspect=1)
    plt.show()
    return 0        

In [123]:
plot_perpendicular_bisector(ax, x[0], y[0], x, y)

0

In [ ]:
#arrange the perpendicular bisectors in the order of nearest last on the stack


